# Introduction

A suite of tests to run before, during and after maintenance windows.

**Usage:**



**Methodology:**

**Supported Collector Functions:**

1. **General**
    1. Reverse DNS lookup for IPs on ARP tables
    1. Vendor lookup for CAM and ARP tables

1. **Cisco**
    1. **NXOS**
        1. Interface status
        1. Trunk / Port-channel status
        1. Spanning-tree (blocked ports, err-disabled ports)
        1. VPC state and status
        1. VLAN database
        1. ARP table (including in VRFs)
        1. CAM table with vendor lookup

    1. **IOS / IOS-XE**
        1. Interface status
        1. Trunk / Port-channel status
        1. Spanning-tree (blocked ports, err-disabled ports, log entries)
        1. VLAN database
        1. ARP table (including in VRFs)
        1. CAM table
        1. Routes (including routes in VRFs)
        1. Routing neighbors
        1. VRFs

1. **ASA**
        1. In Progress
    1. **Meraki**
        1. TBD

1. **F5**
    1. **F5 LTM**
        1. Interface status
        1. Trunk status
        1. VLANs
        1. ARP table
        1. CAM table
        1. Total VIPs
        1. Active VIPs
        1. Total Pools
        1. Active Pools
        1. Total Pool Members
        1. Active Pool Members
        1. NMAP scans of F5 VIPs (for service verification)
        1. Verification of web services behind F5 VIPs

1. **Palo Alto**
    1. ARP table

**Validators**

Note that the output of all collectors can be displayed and reviewed in Jupyter or Excel. However, the goal is to create validators for all collectors.

1. **F5 LTM**
    1. Interface status
    1. Trunk status
    1. Available VIPs
    1. Available Pools
    1. Available Pool Members
    1. NMAP scans of F5 VIPs
    1. Testing connectivity to VIPs using ports 80, 8080, and 443.

# Setup

## Import Modules, Set Pandas Display Options and Get the Path the Notebook Is Being Run From


In [1]:
import ansible
import ansible_runner
import ipaddress
import nmap3
import openpyxl
import os
import pandas as pd
import pathlib
import requests
import sys
import yaml  # Remove after testing is done, because it is imported by the helpers library

from datetime import datetime as dt
from getpass import getpass
from pprint import pprint

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.precision', 3)

nb_path = str(pathlib.Path().resolve())

## Create Base Functions

In [2]:
def set_filename(filepath):
    prefix = dt.now().strftime("%Y-%m-%d_%H%M")
    filename = f'{filepath}/{prefix}.xlsx'
    return filename

### Import the 'helpers' Library and Set the Path to the Playbooks

In [3]:
filepath = input("Enter the path to the Net-Manage repository: ")

if '~' in filepath:
    root_path = os.path.expanduser(filepath)
    sys.path.insert(1, root_path)
else:
    root_path = filepath
    sys.path.insert(1, root_path)
import helpers as hp
play_path = f'{root_path}/playbooks'

Enter the path to the Net-Manage repository:  ~/source/repos/InsightSSG/Net-Manage


## Set the Base Output Filepath

The filename is prefixed with the date and time. For example, if you set the base path to '~/output' then the filename will be *%Y-%m-%d_%H%M.xlsx*

In [4]:
filepath = input('Enter the filepath (do NOT use this directory): ')

if '~' in filepath:
    filepath = filepath.replace('~', os.path.expanduser('~'))

Enter the filepath (do NOT use this directory):  output


## Get Credentials

In [5]:
username, password = hp.get_creds()

Enter the username to login to the devices with:  js_370802
Enter your password:  ········
Confirm your password:  ········


## Create the Dictionary to Store Pre- and Post-check DataFrames

In [6]:
checks = dict()
checks['f5s'] = dict()
checks['nxos'] = dict()

## Create a DataFrame Containing the Tests to Run

### Get the Path to the Spreadsheet and the Name of the Workbook Containing the Tests and Read It Into a DataFrame

In [ ]:
spreadsheet = input('Enter the path to the spreadsheet containing the tests to run: ')
worksheet = input('Enter the name of the worksheet inside the spreadsheet that contains the tests: ')

df_tests = pd.read_excel(spreadsheet, sheet_name=worksheet)
df_tests = df_tests.astype(str)
df_tests.info()
display(df_tests)

## Create a DataFrame Containing the Variables for Each Host Group

In [ ]:
df_vars = hp.ansible_create_vars_df(df_tests)
df_vars.info()
display(df_vars)

# Implementation

## Cisco Pre-checks

### Cisco NXOS Pre-checks

Runs the following tests:
* Get running-config diff
* Get err-disabled interfaces
* Get logs of err-disabled interfaces (to check for interfaces that auto-recovered)
* Get spanning-tree blocked ports
* Get VPC state
* Get VPC consistency
* Get VPC peer-keepalive

**ToDo:**

* Get ARP table
* Get CAM table (?)



    commands = {'diff_command': 'show running-config diff',
                'interface_status_command': 'show interface status err-disabled',
                'logging_command': 'show logging last 9999 | grep "err-disable\|BPDU"',
                'stp_blocked_ports_command': 'show spanning-tree blockedports',
                'vpc_brief_command': 'show vpc brief | grep Po',
                'vpc_consistency_command': 'show vpc consistency-parameters global | grep "Local suspended"',
                'vpc_keepalive_command': 'show vpc peer-keepalive | grep -v "ms\|msec"'}

In [7]:
def cisco_nxos_run_checks(username,
                          password,
                          host_group,
                          play_path,
                          private_data_dir):
    '''
    Run the pre- and post-checks on a Cisco NXOS and adds the results to
    a dictionary. Also creates a list of devices in the host group.
    
    Args:
        username (str):         The username to login to devices
        password (str):         The password to login to devices
        host_group (str):       The inventory host group
        play_path (str):        The path to the playbooks directory
        private_data_dir (str): The path to the Ansible private data directory

    Returns:
        devices (list):    A list of devices in the host group
        check_data (dict): A dictionary containing the pre-checks
    '''
    # Create the base variables
    check_data = dict()
    
    # Define the commands to run
    commands = {'hostname_command': 'show hostname',
                'diff_command': 'show running-config diff',
                'interface_status_command': 'show interface status',
                'err_disabled_command': 'show interface status err-disabled',
                'logging_command': 'show logging last 9999 | grep "err-disable\|BPDU"',
                'stp_blocked_ports_command': 'show spanning-tree blockedports',
                'vpc_state_command': 'show vpc brief | begin "vPC domain id" | end "vPC Peer-link status"',
                'vpc_status_command': 'show vpc brief | begin "vPC status"',
                'vpc_consistency_command': 'show vpc consistency-parameters global | grep "Local suspended"',
                'vpc_keepalive_command': 'show vpc peer-keepalive | grep -v "ms\|msec"',
                'get_arp_table_command': 'show ip arp vrf all | begin "MAC Address"',
                'get_cam_table_command': 'show mac address-table | begin "MAC Address"'}

    # Create the extra variables to pass to Ansible
    extravars = {'username': username,
                 'password': password,
                 'host_group': host_group,
                 'commands': commands}

    # Execute the pre-checks
    runner = ansible_runner.run(private_data_dir='.',
                                playbook=f'{play_path}/cisco_nxos_pre_post_checks.yml',
                                extravars=extravars)
    
    # Parse the output; add the hosts to 'devices' and the command output to 'check_data'
    for event in runner.events:
        if event['event'] == 'runner_on_ok':
            event_data = event['event_data']
            device = event_data['remote_addr']
            
            # Create the dictionary keys for 'check_data'
            if not check_data.get(device):
                check_data[device] = dict()
            
            # Add the command output to 'check_data'
            cmd = event_data['res']['invocation']['module_args']['commands'][0]
            output = event_data['res']['stdout'][0].split('\n')
            check_data[device][cmd] = output

    return check_data

group = 'cts_nexuses'

pre_check_data = cisco_nxos_run_checks(username, password, group, play_path, private_data_dir='.')

[WARNING]: No inventory was parsed, only implicit localhost is available
[WARNING]: provided hosts list is empty, only localhost is available. Note that
the implicit localhost does not match 'all'
[WARNING]: Could not match supplied host pattern, ignoring: cts_nexuses

PLAY [Cisco NXOS Perform Pre- or Post-Checks] **********************************
skipping: no hosts matched

PLAY RECAP *********************************************************************


In [8]:
def nxos_checks_dump(check_data):
    '''
    Formats the output of the 'cisco_nxos_pre_post_checks.yml' playbook and
    returns it as a dictionary.

    Args:
        check_data (dict): The output of the playbooks

    Returns:
        formatted_checks (dict): A nested dictionary, where the value for each
                                 key is the formatted output
    '''

    nxos_devices = list()
    
    check_hostnames = dict()
    check_hostnames['inventory_name'] = list()
    check_hostnames['hostname'] = list()

    check_inf_status = dict()
    check_inf_status['hostname'] = list()
    check_inf_status['Port'] = list()
    check_inf_status['Name'] = list()
    check_inf_status['Status'] = list()
    check_inf_status['Vlan'] = list()
    check_inf_status['Duplex'] = list()
    check_inf_status['Speed'] = list()
    check_inf_status['Type'] = list()

    check_vpc_status = dict()
    check_vpc_status['hostname'] = list()
    check_vpc_status['id'] = list()
    check_vpc_status['Port'] = list()
    check_vpc_status['Status'] = list()
    check_vpc_status['Consistency'] = list()
    check_vpc_status['Reason'] = list()
    check_vpc_status['Active vlans'] = list()

    check_vpc_state = dict()
    check_vpc_state['hostname'] = list()
    check_vpc_state['vPC domain id'] = list()
    check_vpc_state['Peer status'] = list()
    check_vpc_state['vPC keep-alive status'] = list()
    check_vpc_state['Configuration consistency status'] = list()
    check_vpc_state['Per-vlan consistency status'] = list()
    check_vpc_state['Type-2 consistency status'] = list()
    check_vpc_state['vPC role'] = list()
    check_vpc_state['Number of vPCs configured'] = list()
    check_vpc_state['Peer Gateway'] = list()
    check_vpc_state['Peer gateway excluded VLANs'] = list()
    check_vpc_state['Dual-active excluded VLANs'] = list()
    check_vpc_state['Graceful Consistency Check'] = list()
    check_vpc_state['Operational Layer3 Peer-router'] = list()
    check_vpc_state['Auto-recovery status'] = list()

    # Extract the ARP table
    check_arp_table = dict()
    check_arp_table['hostname'] = list()
    check_arp_table['Address'] = list()
    check_arp_table['Age'] = list()
    check_arp_table['MAC Address'] = list()
    check_arp_table['Interface'] = list()

    # Extract the CAM table
    check_cam_table = dict()
    check_cam_table['hostname'] = list()
    check_cam_table['Legend'] = list()
    check_cam_table['VLAN'] = list()
    check_cam_table['MAC Address'] = list()
    check_cam_table['Type'] = list()
    check_cam_table['age'] = list()
    check_cam_table['Secure'] = list()
    check_cam_table['NTFY'] = list()
    check_cam_table['Ports/SWID.SSID.LID'] = list()
    
    # Extract the spanning-tree blocked ports
    check_stp_blocked = dict()
    check_stp_blocked['hostname'] = list()
    check_stp_blocked['Name'] = list()
    check_stp_blocked['Blocked Interfaces List'] = list()
    
    # Extract the total number of blocked STP ports
    check_total_stp_blocked = dict()
    check_total_stp_blocked['hostname'] = list()
    check_total_stp_blocked['Total Blocked Ports'] = list()

    # Extract the total number of err-disabled ports
    check_err_disabled = dict()
    check_err_disabled['hostname'] = list()
    check_err_disabled['Port'] = list()
    check_err_disabled['Name'] = list()
    check_err_disabled['Status'] = list()
    check_err_disabled['Reason'] = list()
    
    for key, value in check_data.items():
        # Extract the hostnames
        nxos_devices.append(key)
        check_hostnames['inventory_name'].append(key)
        check_hostnames['hostname'].append(value['show hostname'][0])

        header = value['show interface status err-disabled'][1]
        pos_port = header.index('Port')
        pos_name = header.index('Name')
        pos_status = header.index('Status')
        pos_reason = header.index('Reason')
        for line in value['show interface status err-disabled'][3:]:
            try:
                port = line[pos_port:pos_name].strip()
                name = line[pos_name:pos_status].strip()
                status = line[pos_status:pos_reason].strip()
                reason = line[pos_reason:].strip()
                check_err_disabled['Port'].append(port)
                check_err_disabled['Name'].append(name)
                check_err_disabled['Status'].append(status)
                check_err_disabled['Reason'].append(reason)
                check_err_disabled['hostname'].append(key)
            except Exception:
                pass
        
        num_err_disabled_ports = len(check_err_disabled.get('Port'))
        
        for line in value['show spanning-tree blockedports'][2:]:
            line = line.split()
            if len(line) == 2:
                check_stp_blocked['hostname'].append(key)
                check_stp_blocked['Name'].append(line[0])
                check_stp_blocked['Blocked Interfaces List'].append(line[1])
            if '(segments)' in line:
                check_total_stp_blocked['hostname'].append(key)
                check_total_stp_blocked['Total Blocked Ports'].append(line[-1])
        
        for line in value['show ip arp vrf all | begin "MAC Address"'][1:]:
            check_arp_table['hostname'].append(key)
            check_arp_table['Address'].append(line.split()[0])
            check_arp_table['Age'].append(line.split()[1])
            check_arp_table['MAC Address'].append(line.split()[2])
            check_arp_table['Interface'].append(line.split()[3])        
        
        for line in value['show mac address-table | begin "MAC Address"'][2:]:
            line = line.split()
            if len(line) == 8:
                check_cam_table['Legend'].append(line[0])
                check_cam_table['VLAN'].append(line[1])
                check_cam_table['MAC Address'].append(line[2])
                check_cam_table['Type'].append(line[3])
                check_cam_table['age'].append(line[4])
                check_cam_table['Secure'].append(line[5])
                check_cam_table['NTFY'].append(line[6])
                check_cam_table['Ports/SWID.SSID.LID'].append(line[7])
                check_cam_table['hostname'].append(key)

        
        
        # Extract the interface statuses
        header = value['show interface status'][1]
        pos = header.index('Status')
        for line in value['show interface status'][3:]:
            # Get the interface
            port = line.split()[0]

            # Get the interface name (this accounts for devices that change the
            # starting position of columns)
            line_prefix = line[:pos].split()
            name = ' '.join(line_prefix[1:])

            # Get the remaining parameters
            line_suffix = line[pos:].split()
            status = line_suffix[0]
            vlan = line_suffix[1]
            duplex = line_suffix[2]
            speed = line_suffix[3]
            try:
                port_type = line_suffix[4]
            except Exception:
                port_type = str()

            check_inf_status['hostname'].append(key)
            check_inf_status['Port'].append(port)
            check_inf_status['Name'].append(name)
            check_inf_status['Status'].append(status)
            check_inf_status['Vlan'].append(vlan)
            check_inf_status['Duplex'].append(duplex)
            check_inf_status['Speed'].append(speed)
            check_inf_status['Type'].append(port_type)

        # Extract the VPC state information
        vpc_state = value['show vpc brief | begin "vPC domain id" | end "vPC Peer-link status"'][:-2]
        if 'CTS-CORE' not in key:
            check_vpc_state['hostname'].append(key)
        for line in vpc_state:
            line = line.split(':')
            line = [l.strip() for l in line]
            check_vpc_state[line[0]].append(line[-1])

        vpc_status = value['show vpc brief | begin "vPC status"'][4:]
        vpc_status = [l for l in vpc_status if len(l.split()) != 1]
        for line in vpc_status:
            line = line.split()
            if len(line) == 6 or len(line) == 8:
                id_ = line[0]
                port = line[1]
                status = line[2]
                if len(line) == 8:
                    consistency = 'Not Applicable'
                    reason = 'Consistency Check Not Performed'
                else:
                    consistency = line[3]
                    reason = line[4]
                active_vlans = line[-1]
                if 'CTS-CORE' not in key:
                    check_vpc_status['hostname'].append(key)
                check_vpc_status['id'].append(id_)
                check_vpc_status['Port'].append(port)
                check_vpc_status['Status'].append(status)
                check_vpc_status['Consistency'].append(consistency)
                check_vpc_status['Reason'].append(reason)
                check_vpc_status['Active vlans'].append(active_vlans)        

    if check_vpc_state.get('Peer Gateway'):
        vpc_in_use = True
    else:
        vpc_in_use = False
                
    formatted_checks = dict()
    formatted_checks['check_arp_table'] = check_arp_table
    formatted_checks['check_cam_table'] = check_cam_table
    formatted_checks['check_err_disabled'] = check_err_disabled
    formatted_checks['num_err_disabled_ports'] = num_err_disabled_ports
    formatted_checks['check_hostnames'] = check_hostnames
    formatted_checks['check_inf_status'] = check_inf_status
    formatted_checks['check_stp_blocked'] = check_stp_blocked
    formatted_checks['check_total_stp_blocked'] = check_total_stp_blocked
    formatted_checks['check_vpc_status'] = check_vpc_status
    formatted_checks['check_vpc_state'] = check_vpc_state
    formatted_checks['vpc_in_use'] = vpc_in_use
                
    return formatted_checks


pre_formatted_checks = nxos_checks_dump(pre_check_data)
pre_check_arp_table = pre_formatted_checks['check_arp_table']
pre_check_cam_table = pre_formatted_checks['check_cam_table']
pre_check_err_disabled = pre_formatted_checks['check_err_disabled']
pre_check_num_err_disabled_ports = pre_formatted_checks['num_err_disabled_ports']
pre_check_hostnames = pre_formatted_checks['check_hostnames']
pre_check_inf_status = pre_formatted_checks['check_inf_status']
pre_check_stp_blocked = pre_formatted_checks['check_stp_blocked']
pre_check_total_stp_blocked = pre_formatted_checks['check_total_stp_blocked']
pre_check_vpc_status = pre_formatted_checks['check_vpc_status']
pre_check_vpc_state = pre_formatted_checks['check_vpc_state']
vpc_in_use = pre_formatted_checks['vpc_in_use']

df_pre_arp_table = pd.DataFrame.from_dict(pre_check_arp_table)
del df_pre_arp_table['Age']
# display(df_pre_arp_table)

df_pre_hostnames = pd.DataFrame.from_dict(pre_check_hostnames)
checks['nxos']['hostnames'] = df_pre_hostnames
display(df_pre_hostnames)

if pre_check_num_err_disabled_ports:
    df_pre_err_disabled = pd.DataFrame.from_dict(pre_check_err_disabled)
    checks['nxos']['err_disabled_ports'] = df_pre_err_disabled
    display(df_pre_err_disabled)

df_pre_check_stp_blocked = pd.DataFrame.from_dict(pre_check_stp_blocked)
checks['nxos']['stp_blocked_ports'] = df_pre_check_stp_blocked
# display(df_pre_check_stp_blocked)

df_pre_check_total_stp_blocked = pd.DataFrame.from_dict(pre_check_total_stp_blocked)
checks['nxos']['total_stp_blocked_ports'] = df_pre_check_total_stp_blocked
# display(df_pre_check_total_stp_blocked)

df_pre_inf_status = pd.DataFrame.from_dict(pre_check_inf_status)
checks['nxos']['interface_statuses'] = df_pre_inf_status
# display(df_pre_inf_status)

df_arp_table = pd.DataFrame.from_dict(pre_check_arp_table)
checks['nxos']['arp_table'] = df_arp_table

df_pre_cam_table = pd.DataFrame.from_dict(pre_check_cam_table)
checks['nxos']['cam_table'] = df_pre_cam_table
# display(df_pre_cam_table)

if vpc_in_use:
    df_pre_vpc_state = pd.DataFrame.from_dict(pre_check_vpc_state)
    checks['nxos']['vpc_state'] = df_pre_vpc_state
    display(df_pre_vpc_state)

    df_pre_vpc_status = pd.DataFrame.from_dict(pre_check_vpc_status)
    checks['nxos']['vpc_status'] = df_pre_vpc_status
    display(df_pre_vpc_status)

,inventory_name,hostname
0,CTS-CORE-SWT1,CTS-CORE-SWT1.mmi.local
1,CTS-CORE-SWT2,CTS-CORE-SWT2.mmi.local
2,CTS-ACDC-SWT1,CTS-ACDC-SWT1.mmi.local
3,CTS-ACDC-SWT2,CTS-ACDC-SWT2.mmi.local


,hostname,Port,Name,Status,Reason
0,CTS-ACDC-SWT2,Eth177/1/6,POVESXI002,linkFlapErr,linkFlapErrDisabled


,hostname,vPC domain id,Peer status,vPC keep-alive status,Configuration consistency status,Per-vlan consistency status,Type-2 consistency status,vPC role,Number of vPCs configured,Peer Gateway,Peer gateway excluded VLANs,Dual-active excluded VLANs,Graceful Consistency Check,Operational Layer3 Peer-router,Auto-recovery status
0,CTS-ACDC-SWT1,998,peer adjacency formed ok,peer is alive,success,success,success,"secondary, operational primary",17,Enabled,-,-,Enabled,Disabled,Enabled (timeout = 240 seconds)
1,CTS-ACDC-SWT2,998,peer adjacency formed ok,peer is alive,success,success,success,"primary, operational secondary",17,Enabled,-,-,Enabled,Disabled,Enabled (timeout = 240 seconds)


,hostname,id,Port,Status,Consistency,Reason,Active vlans
0,CTS-ACDC-SWT1,11,Po11,up,success,success,"1-4,6-23,27"
1,CTS-ACDC-SWT1,12,Po12,up,success,success,"1-4,6-23,27"
2,CTS-ACDC-SWT1,513,Po513,up,success,success,"44-45,102,9"
3,CTS-ACDC-SWT1,514,Po514,up,success,success,"44-45,102,9"
4,CTS-ACDC-SWT1,515,Po515,up,success,success,"44-45,102,9"
5,CTS-ACDC-SWT1,516,Po516,up,success,success,"44-45,102,9"
6,CTS-ACDC-SWT1,517,Po517,up,success,success,"44-45,102,9"
7,CTS-ACDC-SWT1,518,Po518,up,success,success,"44-45,102,9"
8,CTS-ACDC-SWT1,519,Po519,up,success,success,"44-45,102,9"
9,CTS-ACDC-SWT1,520,Po520,up,success,success,"44-45,102,9"


## F5 Pre-checks

### Implementation: Run Pre-checks

In [9]:
group = 'f5_ltm_cts'

In [10]:
cmds = {'show_hostname': 'list sys global-settings hostname | grep hostname',
        'sync_status': 'show cm sync-status | grep Status | grep -v CM',
        'sys_version': 'show sys version | grep Version | grep -v Sys',
        'total_vips': 'list ltm virtual /*/* | grep -c "ltm virtual"',
        'vip_availability': 'show ltm virtual /*/* | grep "Ltm::\|Availability"',
        'available_vips': 'show ltm virtual /*/* | grep Availability | grep -c available',
        'vip_destinations': 'show ltm virtual /*/* | grep "Virtual Server\|Availability\|Destination"',
        'active_pool_members': 'show ltm pool /*/* | grep "Current Active Members"',
        'pool_member_availability': 'show ltm pool /*/* members detail | grep "Ltm::Pool\|Availability"',
        'get_arp_table': 'show net arp | grep -v "\-\-\-\|Net::Arp\|HWaddress"',  # TODO: Does this command display ARP entries for all partitions?
        'get_cam_table': 'show sys mac-address | grep "mac-address"',
        'show_net_interface': 'show net interface | grep -v "\-\-\-\|Net::Interface\|Bits\|Out"',
        'list_net_interface': 'list net interface',
        # 'get_irules': 'list ltm rule /*/*'
       }

extravars = {'host_group': group,
             'user': username,
             'password': password,
             'validate_certs': 'no',
             'ansible_timeout': '200'}

for cmd in cmds:
    extravars[cmd] = cmds[cmd]
    
devices = list()

results = dict()

runner = ansible_runner.run(private_data_dir='.',
                            playbook=f'{play_path}/f5_pre_post_checks.yml',
                            extravars=extravars)

print(f'{runner.status}: {runner.rc}')

results = dict()

for event in runner.events:
    if event['event'] == 'runner_on_ok':
        # Parse the event to extract the data
        event_data = event['event_data']
        host = event_data['host']
        cmd = event_data['res']['invocation']['module_args']['commands'].pop()
        result = event_data['res']['stdout'][0]

        # If a nested dictionary for the host does not yet exist, then create it
        if not results.get(host):
            results[host] = dict()

        # Map the command to the corresponding key in the 'cmds' dictionary and add
        # it to the result.
        for key, value in cmds.items():
            if value == cmd:
                results[host][key] = result
                break

        # Add the host to the 'devices' list
        if host not in devices:
            devices.append(host)



PLAY [F5 Health Check] *********************************************************

TASK [show hostname] ***********************************************************
ok: [f510 -> localhost]
ok: [f509 -> localhost]

TASK [get sync status] *********************************************************
ok: [f510 -> localhost]
ok: [f509 -> localhost]

TASK [get sys version] *********************************************************
ok: [f510 -> localhost]
ok: [f509 -> localhost]

TASK [show net interface] ******************************************************
ok: [f510 -> localhost]
ok: [f509 -> localhost]

TASK [list net interface] ******************************************************
ok: [f510 -> localhost]
ok: [f509 -> localhost]

TASK [get number of VIPs] ******************************************************
ok: [f510 -> localhost]
ok: [f509 -> localhost]

TASK [get VIP availability] ****************************************************
ok: [f510 -> localhost]
ok: [f509 -> localhost]

TASK [g

In [11]:
# Create a dictionary to store the pre-check data
pre_checks = dict()
pre_checks['device'] = list()
for cmd in cmds:
    pre_checks[cmd] = list()

# Remove dictionary keys that we do not need for 'df_pre'
excluded_cmds = ['vip_availability',
                 'pool_member_availability',
                 'get_arp_table',
                 'get_cam_table',
                 # 'show_net_interface',
                 'vip_destinations',
                 'list_net_interface'
                 ]
# for cmd in excluded_cmds:
#     del pre_checks[cmd]

# Parse the 'results' dictionary and add the applicable output to 'df_pre'
for device in devices:
    for key, value in results[device].items():
        if key == 'show_hostname':
            # pprint(value)
            hostname = value.split()[-1]
        if key == 'sync_status':
            sync_status = value.split('Status')[-1].strip()
        if key == 'sys_version':
            sys_version = value.split()[-1]
        if key == 'total_vips':
            total_vips = value
        if key == 'available_vips':
            available_vips = value

        if key == 'vip_destinations':
            vip_destinations = dict()
            vip_destinations['hostname'] = list()
            vip_destinations['name'] = list()
            vip_destinations['availability'] = list()
            vip_destinations['destination'] = list()
            vip_destinations['port'] = list()
            for line in value.split('\n'):
                if line.split()[0] == 'Ltm::Virtual':
                    vip_destinations['hostname'].append(device)
                    name = line.split()[-1]
                    vip_destinations['name'].append(name)
                if line.split()[0] == 'Availability':
                    availability = line.split()[-1]
                    vip_destinations['availability'].append(availability)
                if line.split()[0] == 'Destination':
                    destination = line.split()[-1].split(':')[0]
                    vip_destinations['destination'].append(destination)
                    port = line.split()[-1].split(':')[-1]
                    vip_destinations['port'].append(port)
            df_vip_destinations = pd.DataFrame.from_dict(vip_destinations)
            
        if key == 'active_pool_members':
            num_members = 0
            for line in value.split('\n'):
                num_members += int(line.split()[-1])
            active_pool_members = str(num_members)
        if key == 'show_net_interface':
            num_up_interfaces = 0
            for line in value.split('\n'):
                if line.split()[1] == 'up':
                    num_up_interfaces += 1
    
    # Add the results to the 'pre_checks' dictionary
    pre_checks['device'].append(device)
    pre_checks['show_hostname'].append(hostname)
    pre_checks['sync_status'].append(sync_status)
    pre_checks['sys_version'].append(sys_version)
    pre_checks['total_vips'].append(total_vips)
    pre_checks['available_vips'].append(available_vips)
    pre_checks['active_pool_members'].append(active_pool_members)
    pre_checks['show_net_interface'].append(num_up_interfaces)

# Try to delete the 'df_pre' dataframe. This is to avoid corruption if it is recreated
try:
    del df_pre
except Exception:
    pass

# Remove from 'pre_checks' any keys that are in excluded_cmds
to_delete = list()
for key, value in pre_checks.items():
    if key in excluded_cmds:
        to_delete.append(key)
for cmd in to_delete:
    del pre_checks[cmd]

checks['f5s']['summary'] = pd.DataFrame.from_dict(pre_checks)

display(checks['f5s']['summary'])

,device,show_hostname,sync_status,sys_version,total_vips,available_vips,active_pool_members,show_net_interface
0,f510,f510.mmi.local,In Sync,13.1.5,227,121,386,5
1,f509,f509.mmi.local,In Sync,13.1.5,227,121,386,5


### Create Additional Dataframes

In [12]:
# Create a dataframe for individual pool member states
pre_member_states = dict()
pre_member_states['device'] = list()
pre_member_states['pool_name'] = list()
pre_member_states['pool_member'] = list()
pre_member_states['pool_member_state'] = list()

# Iterate through 'results', adding pool members states to 'pre_member_states'
for device in devices:
    member_states = results[device]['pool_member_availability'].split('\n')
    for line in member_states:
        if 'Ltm::Pool:' in line:
            pool_name = line.split('Ltm::Pool: ')[-1]
            try:
                pos = member_states.index(line)+1
                while 'Ltm::Pool:' not in member_states[pos]:
                    if '| Ltm::Pool Member:' in member_states[pos]:
                        pool_member = member_states[pos].split('Ltm::Pool Member: ')[-1]
                        pool_member_state = member_states[pos+1].split(':')[-1].strip()

                        pre_member_states['device'].append(device)
                        pre_member_states['pool_name'].append(pool_name)
                        pre_member_states['pool_member'].append(pool_member)
                        pre_member_states['pool_member_state'].append(pool_member_state)
                    pos += 1
            except Exception as e:
                if str(e) == 'list index out of range':
                    break
                else:
                    print(str(e))
                    
# Try to delete 'df_pre_member_state'
try:
    del df_pre_member_state
except Exception:
    pass

# Create 'df_pre_member_state'
df_pre_member_state = pd.DataFrame.from_dict(pre_member_states)
checks['f5s']['pool_member_states'] = df_pre_member_state

display(df_pre_member_state.head(10))

# Create a dataframe for individual VIP states
pre_vip_states = dict()
pre_vip_states['device'] = list()
pre_vip_states['vip_name'] = list()
pre_vip_states['vip_state'] = list()

# Iterate through 'results', adding VIP states to 'pre_vip_states'
for device in devices:
    vip_availability = results[device]['vip_availability'].split('\n')
    for line in vip_availability:
        if 'Ltm::Virtual Server:' in line:
            # Get the position of the line in the list
            pos = vip_availability.index(line)

            # Use the position to get the VIP availability
            vip_state = vip_availability[pos+1].split()[-1]
            
            # Extract the VIP name from the line
            vip_name = line.split('Ltm::Virtual Server: ')[-1]

            # Add the data to 'pre_vip_states'
            pre_vip_states['device'].append(device)
            pre_vip_states['vip_name'].append(vip_name)
            pre_vip_states['vip_state'].append(vip_state)
            
# Try to delete 'df_pre_vip_state'
try:
    del df_pre_vip_state
except Exception:
    pass

# Create 'df_pre_vip_state'
df_pre_vip_state = pd.DataFrame.from_dict(pre_vip_states)
checks['f5s']['vip_states'] = df_pre_vip_state
display(df_pre_vip_state.head(10))

# Create a dataframe for the ARP table
# Define the dataframe columns
cols = ['name',
        'address',
        'hw_address',
        'vlan',
        'expire_in_sec',
        'status']



# Create a dictionary to store the ARP table data
pre_arp = dict()
pre_arp['device'] = list()
for c in cols:
    pre_arp[c] = list()

# Iterate through 'results', adding ARP table data to 'pre_arp'
for device in devices:
    arp_table = results[device]['get_arp_table'].split('\n')
    for line in arp_table:
        pre_arp['device'].append(device)
        pre_arp['name'].append(line.split()[0])
        pre_arp['address'].append(line.split()[1])
        pre_arp['hw_address'].append(line.split()[2])
        pre_arp['vlan'].append(line.split()[3])
        pre_arp['expire_in_sec'].append(line.split()[4])
        pre_arp['status'].append(line.split()[5])
            
# Try to delete 'pre_arp'
try:
    del df_pre_arp
except Exception:
    pass

# Create 'df_pre_arp'
df_pre_arp = pd.DataFrame.from_dict(pre_arp)
checks['f5s']['arp_table'] = df_pre_arp
display(df_pre_arp.head(10))




# Create a dataframe for the CAM table
cols = ['device',
        'entry',
        'component',
        'object_id',
        'property']

# Create a dictionary to store the CAM table data
pre_cam = dict()
pre_cam['device'] = list()
for c in cols:
    pre_cam[c] = list()

# Iterate through 'results', adding ARP table data to 'pre_arp'
for device in devices:
    cam_table = results[device]['get_cam_table'].split('\n')
    for line in cam_table:
        pre_cam['device'].append(device)
        pre_cam['entry'].append(line.split()[0])
        pre_cam['component'].append(line.split()[1])
        pre_cam['object_id'].append(line.split()[2])
        pre_cam['property'].append(line.split()[3])
            
# Try to delete 'df_pre_cam'
try:
    del df_pre_cam
except Exception:
    pass

# Create 'df_pre_cam'
df_pre_cam = pd.DataFrame.from_dict(pre_cam)
checks['f5s']['cam_table'] = df_pre_cam
display(df_pre_cam.head(10))

,device,pool_name,pool_member,pool_member_state
0,f510,/BANFIELD.COM-PROD/BANFIELD.COM-80,M1DMZNET03:80,offline
1,f510,/BANFIELD.COM-PROD/BANFIELD.COM-80,M1DMZNET04:80,offline
2,f510,/BANFIELD.COM-PROD/BANFIELD.COM-80,M1DMZNET11:80,available
3,f510,/BANFIELD.COM-PROD/BANFIELD.COM-80,M1DMZNET12:80,available
4,f510,/BANFIELD.COM-PROD/BANFIELD.COM-443,M1DMZNET03:443,offline
5,f510,/BANFIELD.COM-PROD/BANFIELD.COM-443,M1DMZNET04:443,offline
6,f510,/BANFIELD.COM-PROD/BANFIELD.COM-443,M1DMZNET11:443,available
7,f510,/BANFIELD.COM-PROD/BANFIELD.COM-443,M1DMZNET12:443,available
8,f510,/BANFIELD.COM-PROD/BANFIELD.COM-443-TESTPOOL,M1DMZNET11:443,available
9,f510,/BANFIELD.COM-PROD/BANFIELD.COM-443-TESTPOOL,M1DMZNET12:443,available


,device,vip_name,vip_state
0,f510,/BANFIELD.COM-PROD/BANFIELD.COM-443,available
1,f510,/BANFIELD.COM-PROD/BANFIELD.COM-8080,available
2,f510,/BANFIELD.COM-PROD/BANFIELD.COM-8082,available
3,f510,/BANFIELD.COM-PROD/BANFIELD.COM-8089,available
4,f510,/BANFIELD.COM-PROD/BANFIELD.COM-8091,available
5,f510,/BANFIELD.COM-PROD/BANFIELD.COM-8099,available
6,f510,/BANFIELD.COM-PROD/BANFIELD.COM-8443,available
7,f510,/BANFIELD.COM-PROD/BANFIELD.COM-HTTP,available
8,f510,/BANFIELD.COM-PROD/BANFIELDPETEXPRESS.COM,available
9,f510,/BANFIELD.COM-PROD/BANFIELDPETEXPRESS.COM-80,unknown


,device,name,address,hw_address,vlan,expire_in_sec,status
0,f510,1.1.1.2,1.1.1.2,00:94:a1:81:d8:2e,/Common/VLAN_3900_F5_HA_CONFIGSYNC,243,resolved
1,f510,2.2.2.2,2.2.2.2,00:94:a1:81:d8:2f,/Common/VLAN_3901_F5_HA_MIRROR,113,resolved
2,f510,66.193.107.254,66.193.107.254,b4:0c:25:e5:40:01,/Common/VLAN975,275,resolved
3,f510,172.21.44.21,172.21.44.21,00:1d:d8:b7:1c:1b,/Common/SVRS-WIN-44,87,resolved
4,f510,172.21.44.24,172.21.44.24,00:1d:d8:b7:1c:14,/Common/SVRS-WIN-44,278,resolved
5,f510,172.21.44.30,172.21.44.30,incomplete,/Common/SVRS-WIN-44,0,unknown
6,f510,172.21.44.43,172.21.44.43,80:30:e0:2c:f6:6c,/Common/SVRS-WIN-44,259,resolved
7,f510,172.21.44.44,172.21.44.44,80:30:e0:2c:88:40,/Common/SVRS-WIN-44,257,resolved
8,f510,172.21.44.46,172.21.44.46,incomplete,/Common/SVRS-WIN-44,0,unknown
9,f510,172.21.44.47,172.21.44.47,00:50:56:be:09:dd,/Common/SVRS-WIN-44,74,resolved


,device,entry,component,object_id,property
0,f510,00:00:0c:07:ac:01,net,arp,172.21.100.254
1,f510,00:1d:d8:b7:1c:1b,net,arp,172.21.44.21
2,f510,00:1d:d8:b7:1c:13,net,arp,172.21.71.180
3,f510,00:1d:d8:b7:1c:14,net,arp,172.21.44.24
4,f510,00:1d:d8:b7:1c:24,net,arp,172.21.242.42
5,f510,00:15:5d:01:7d:01,net,arp,172.21.71.133
6,f510,00:15:5d:01:7d:02,net,arp,172.21.71.189
7,f510,00:15:5d:01:44:0d,net,arp,172.21.71.60
8,f510,00:15:5d:01:44:1e,net,arp,172.21.71.61
9,f510,00:15:5d:01:44:03,net,arp,172.21.71.51


### Use NMAP To Scan Available VIP Ports

In [13]:
destinations_all = df_vip_destinations['destination'].to_list()
destinations_any = df_vip_destinations.loc[df_vip_destinations['port'] == 'any']['destination'].to_list()

ports = df_vip_destinations['port'].to_list()
ports = [p for p in ports if p != 'any']
ports = list(set(ports))

top_port = 0
for p in ports:
    if int(p) > top_port:
        top_port = int(p)

len(df_vip_destinations), len(destinations_all), len(destinations_any), len(ports), top_port

checks['f5s']['vip_destinations'] = df_vip_destinations

display(checks['f5s']['vip_destinations'].head(10))

,hostname,name,availability,destination,port
0,f509,/BANFIELD.COM-PROD/BANFIELD.COM-443,available,66.193.107.148,443
1,f509,/BANFIELD.COM-PROD/BANFIELD.COM-8080,available,66.193.107.148,8080
2,f509,/BANFIELD.COM-PROD/BANFIELD.COM-8082,available,66.193.107.148,8082
3,f509,/BANFIELD.COM-PROD/BANFIELD.COM-8089,available,66.193.107.148,8089
4,f509,/BANFIELD.COM-PROD/BANFIELD.COM-8091,available,66.193.107.148,8091
5,f509,/BANFIELD.COM-PROD/BANFIELD.COM-8099,available,66.193.107.148,8099
6,f509,/BANFIELD.COM-PROD/BANFIELD.COM-8443,available,66.193.107.148,8443
7,f509,/BANFIELD.COM-PROD/BANFIELD.COM-HTTP,available,66.193.107.148,80
8,f509,/BANFIELD.COM-PROD/BANFIELDPETEXPRESS.COM,available,66.193.107.240,443
9,f509,/BANFIELD.COM-PROD/BANFIELDPETEXPRESS.COM-80,unknown,66.193.107.240,80


In [14]:
nmap = nmap3.Nmap()
print('Scanning destinations that have defined ports')
results_unique = nmap.scan_top_ports(' '.join(destinations_all),
                                     args=f'-np {",".join(ports)}',
                                     default=top_port)

print('Doing a "top ports" scan of ports with a destination of "any"')
results_any = nmap.scan_top_ports(' '.join(destinations_any),
                                  args='-n')

def remove_invalid_keys(result):
    invalid_keys = list()
    for key in result:
        try:
            if ipaddress.ip_address(key):
                pass
        except Exception:
            invalid_keys.append(key)
    for key in invalid_keys:
        del result[key]
    return result

results_unique = remove_invalid_keys(results_unique)
results_any = remove_invalid_keys(results_any)
      
len(df_vip_destinations), len(results_unique), len(results_any)

Scanning destinations that have defined ports
Doing a "top ports" scan of ports with a destination of "any"


(227, 110, 16)

In [15]:
open_ports = list()
closed_ports = list()

scan_dict = dict()
for key, value in results_unique.items():
    scan_dict[key] = dict()
    scan_dict[key]['open'] = list()
    scan_dict[key]['closed'] = list()
    
    for item in value['ports']:
        if item.get('state'):
            port = item.get('portid')
            state = item.get('state')
            if state == 'open':
                scan_dict[key]['open'].append(port)
            else:
                scan_dict[key]['closed'].append(port)

for key, value in results_any.items():
    if not scan_dict.get(key):
        scan_dict[key] = dict()
        scan_dict[key]['open'] = list()
        scan_dict[key]['closed'] = list()
    
    for item in value['ports']:
        if item.get('state'):
            state = item.get('state')
            port = item.get('portid')
            
            if state == 'open':
                scan_dict[key]['open'].append(port)
            else:
                scan_dict[key]['closed'].append(port)            
        
for idx, row in df_vip_destinations.iterrows():
    destination = row['destination']
    if scan_dict.get(destination):
        p_open = ' '.join(scan_dict[destination].get('open'))
        p_closed = ' '.join(scan_dict[destination].get('closed'))
        
    else:
        p_open = 'unknown'
        p_closed = 'unknown'
        
    open_ports.append(p_open)
    closed_ports.append(p_closed)

df_vip_destinations['open ports'] = open_ports
df_vip_destinations['closed ports'] = closed_ports
    
len(df_vip_destinations), len(open_ports), len(closed_ports)

(227, 227, 227)

##### Troubleshoot: Why is df_vip_destinations just getting results from one device? Or is it not updating the device name?

In [16]:
df_http = df_vip_destinations.loc[df_vip_destinations['availability'] == 'available'].copy()

df_available_http_vips = df_http.loc[(df_http['port'] == '80') |
                                     (df_http['port'] == '443') | 
                                     (df_http['port'] == '8080')]

checks['f5s']['available_http_vips'] = df_available_http_vips

display(df_available_http_vips)

,hostname,name,availability,destination,port,open ports,closed ports
0,f509,/BANFIELD.COM-PROD/BANFIELD.COM-443,available,66.193.107.148,443,80 443 8080 8082 8089 8091 8099 8443,1521 8050 8892
1,f509,/BANFIELD.COM-PROD/BANFIELD.COM-8080,available,66.193.107.148,8080,80 443 8080 8082 8089 8091 8099 8443,1521 8050 8892
7,f509,/BANFIELD.COM-PROD/BANFIELD.COM-HTTP,available,66.193.107.148,80,80 443 8080 8082 8089 8091 8099 8443,1521 8050 8892
8,f509,/BANFIELD.COM-PROD/BANFIELDPETEXPRESS.COM,available,66.193.107.240,443,80 443,
11,f509,/BANFIELD.COM-PROD/LAUNCHPAD-443,available,66.193.107.134,443,443,
13,f509,/BANFIELD.COM-PROD/MMIHOLDINGS.COM-443,available,66.193.107.222,443,80 443,
16,f509,/BANFIELD.COM-PROD/POINT-443,available,66.193.107.224,443,443,8892
18,f509,/BANFIELD.COM-PROD/POSAPP-8076,available,172.21.133.91,443,443 8080 8091 8098,
20,f509,/BANFIELD.COM-PROD/POSAPP-8080,available,172.21.133.91,8080,443 8080 8091 8098,
24,f509,/BANFIELD.COM-PROD/STATEOFPH-80,available,66.193.107.171,80,80 443,


### Check if Web Site Responds

In [17]:
http_responses = dict()
http_responses['name'] = list()
http_responses['destination'] = list()
http_responses['size'] = list()
http_responses['error'] = list()

for idx, row in df_available_http_vips.iterrows():
    destination = row['destination']
    website = f'http://{destination}'
    try:
        r = requests.get(website, timeout=15)
        http_responses['name'].append(row['name'])
        http_responses['destination'].append(destination)
        http_responses['size'].append(len(r.content))
        error = str(0)
        http_responses['error'].append(error)
    except Exception as e:
        http_responses['name'].append(row['name'])
        http_responses['destination'].append(destination)
        http_responses['size'].append(0)
        error = str(e)
        http_responses['error'].append(error)

df_http_responses = pd.DataFrame.from_dict(http_responses)

checks['f5s']['http_responses'] = df_http_responses

display(df_http_responses)

,name,destination,size,error
0,/BANFIELD.COM-PROD/BANFIELD.COM-443,66.193.107.148,5816,0
1,/BANFIELD.COM-PROD/BANFIELD.COM-8080,66.193.107.148,5816,0
2,/BANFIELD.COM-PROD/BANFIELD.COM-HTTP,66.193.107.148,5816,0
3,/BANFIELD.COM-PROD/BANFIELDPETEXPRESS.COM,66.193.107.240,0,"('Connection aborted.', ConnectionResetError(1..."
4,/BANFIELD.COM-PROD/LAUNCHPAD-443,66.193.107.134,0,"HTTPConnectionPool(host='66.193.107.134', port..."
5,/BANFIELD.COM-PROD/MMIHOLDINGS.COM-443,66.193.107.222,0,"HTTPSConnectionPool(host='66.193.107.222', por..."
6,/BANFIELD.COM-PROD/POINT-443,66.193.107.224,0,"HTTPConnectionPool(host='66.193.107.224', port..."
7,/BANFIELD.COM-PROD/POSAPP-8076,172.21.133.91,0,"HTTPConnectionPool(host='172.21.133.91', port=..."
8,/BANFIELD.COM-PROD/POSAPP-8080,172.21.133.91,0,"HTTPConnectionPool(host='172.21.133.91', port=..."
9,/BANFIELD.COM-PROD/STATEOFPH-80,66.193.107.171,0,"HTTPConnectionPool(host='66.193.107.171', port..."


# Save the Checks to a Spreadsheet

In [18]:
filename = set_filename(filepath)
filename

with pd.ExcelWriter(filename) as writer:
    for key, value in checks.items():
        for k, v in value.items():
            sheet_name = f'{key}_{k}'
            v.to_excel(writer, sheet_name=sheet_name, index=False)
            
print(f'File saved to {filename}')

File saved to output/2022-08-21_0414.xlsx


## Compare Spreadsheets

In [19]:
comp_files = input('Enter the names (comma-delimited) of two spreadsheets to compare: ')
comp_files = [f.strip() for f in comp_files.split(',')]
comp_files

Enter the names (comma-delimited) of two spreadsheets to compare:  output/pre_checks_2022-08-20_2047.xlsx,output/2022-08-21_0414.xlsx


['output/pre_checks_2022-08-20_2047.xlsx', 'output/2022-08-21_0414.xlsx']

In [20]:
try:
    del dict_1
except Exception:
    pass

try:
    del dict_2
except Exception:
    pass

dict_1 = pd.read_excel(comp_files[0], sheet_name=None)
dict_2 = pd.read_excel(comp_files[1], sheet_name=None)

In [21]:
status_dict = dict()
status_dict['f5s_pool_member_states'] = list()
status_dict['f5s_vip_states'] = list()
status_dict['f5s_arp_table'] = list()
status_dict['f5s_vip_destinations'] = list()
status_dict['f5s_available_http_vips'] = list()

for key, value in dict_1.items():
    try:
        del df_1
    except Exception:
        pass
    try:
        del df_2
    except Exception:
        pass

    df_1 = value.copy()
    df_2 = dict_2[key].copy()
    
    if key == 'f5s_summary':
        display(df_1)
        display(df_2)
    
    if key == 'f5s_pool_member_states':
        status_dict[key].append(len(df_1.loc[df_1['pool_member_state'] == 'available']))
        status_dict[key].append(len(df_2.loc[df_2['pool_member_state'] == 'available']))                     
        # print(key, len(df_1.loc[df_1['pool_member_state'] == 'available']), len(df_2.loc[df_2['pool_member_state'] == 'available']))
        
    if key == 'f5s_vip_states':
        status_dict[key].append(len(df_1.loc[df_1['vip_state'] == 'available']))
        status_dict[key].append(len(df_2.loc[df_2['vip_state'] == 'available']))
        # print(key, len(df_1.loc[df_1['vip_state'] == 'available']), len(df_2.loc[df_2['vip_state'] == 'available']))
        
    if key == 'f5s_arp_table':
        status_dict[key].append(len(df_1.loc[df_1['status'] == 'resolved']))
        status_dict[key].append(len(df_2.loc[df_2['status'] == 'resolved']))
        # print(key, len(df_1.loc[df_1['status'] == 'resolved']), len(df_2.loc[df_2['status'] == 'resolved']))
        
    if key == 'f5s_vip_destinations':
        status_dict[key].append(len(df_1.loc[df_1['availability'] == 'available']))
        status_dict[key].append(len(df_2.loc[df_2['availability'] == 'available']))
        # print(key, len(df_1.loc[df_1['availability'] == 'available']), len(df_2.loc[df_2['availability'] == 'available']))

    if key == 'f5s_available_http_vips':
        status_dict[key].append(len(df_1.loc[df_1['availability'] == 'available']))
        status_dict[key].append(len(df_2.loc[df_2['availability'] == 'available']))

df_status = pd.DataFrame.from_dict(status_dict).T
# df_status.rename(columns = {'0': comp_files[0], '1': comp_files[1]}, inplace=True)
df_status.columns = [comp_files[0], comp_files[1]]

display(df_status)

,device,show_hostname,sync_status,sys_version,total_vips,available_vips,active_pool_members,show_net_interface
0,f510,f510.mmi.local,In Sync,13.1.5,227,121,385,5
1,f509,f509.mmi.local,In Sync,13.1.5,227,121,385,5


,device,show_hostname,sync_status,sys_version,total_vips,available_vips,active_pool_members,show_net_interface
0,f510,f510.mmi.local,In Sync,13.1.5,227,121,386,5
1,f509,f509.mmi.local,In Sync,13.1.5,227,121,386,5


,output/pre_checks_2022-08-20_2047.xlsx,output/2022-08-21_0414.xlsx
f5s_pool_member_states,794,796
f5s_vip_states,242,242
f5s_arp_table,485,499
f5s_vip_destinations,121,121
f5s_available_http_vips,32,32


In [ ]:
for key, value in dict_1.items():
    print(key)

In [ ]:
for key, value in dict_1.items():
    print(key)
    print('\n')
    print(value.info())
    print(dict_2[key].info())
    display(value.head(10))
    display(dict_2[key].head(10))
    print('\n')
    print('\n')